# Preprocess Data

In [1]:
import os
import random
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

In [2]:
train_data_path = "./train_preliminary/"
train_final_data_path = "./train_semi_final/"
test_data_path = "./test/"
save_path = './processed_data'
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [3]:
df_ad = pd.concat([pd.read_csv(os.path.join(train_data_path, "ad.csv")), pd.read_csv(os.path.join(train_final_data_path, "ad.csv")), pd.read_csv(os.path.join(test_data_path, "ad.csv"))])

df_click = pd.concat([pd.read_csv(os.path.join(train_data_path, "click_log.csv")), pd.read_csv(os.path.join(train_final_data_path, "click_log.csv")), pd.read_csv(os.path.join(test_data_path, "click_log.csv"))])
                            
df_train_user = pd.concat([pd.read_csv(os.path.join(train_data_path, "user.csv")), pd.read_csv(os.path.join(train_final_data_path, "user.csv"))])

In [4]:
# 复赛和初赛、训练集和测试集点击的广告有重复部分，去重
df_ad = df_ad.drop_duplicates()

## 处理空值

In [5]:
df_ad[df_ad=="\\N"] = np.nan

In [6]:
df_ad.fillna(0, inplace=True)

In [7]:
df_ad = df_ad.astype(int)

In [8]:
(df_ad == 0).sum() 

creative_id               0
ad_id                     0
product_id          1575509
product_category          0
advertiser_id             0
industry             183159
dtype: int64

In [9]:
# 缺失值最终填充为最大值，0留给pad
df_ad.loc[df_ad['product_id']==0, 'product_id'] = df_ad['product_id'].max() + 1
df_ad.loc[df_ad['industry']==0, 'industry'] = df_ad['industry'].max() + 1

## 将广告的信息合并到用户点击数据表中

In [10]:
df_click = df_click.merge(df_ad, on='creative_id')

## 排序，确保后期分组时仍然按时间有序


In [11]:
df_click.sort_values(by=['user_id', 'time'], inplace=True)

In [12]:
df_click = df_click.astype(np.int32)

In [13]:
df_click

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
71892857,20,1,877468,1,773445,44315,5,29455,106
101809561,20,1,209778,1,188507,136,2,9702,6
117324299,20,1,821396,1,724607,44315,5,7293,326
99387603,39,1,1683713,1,1458878,44315,5,14668,326
42203506,40,1,122032,1,109959,1334,2,11411,336
...,...,...,...,...,...,...,...,...,...
44609273,75,4000000,3596158,1,3096233,44315,18,36668,36
115480909,75,4000000,3642395,1,3135640,1076,2,18422,45
12280290,76,4000000,366858,1,331268,1036,2,36890,319
51542431,76,4000000,3333680,1,2868147,1469,2,32830,21


## 按user_id分组处理数据，将每个用户的点击序列聚集到一起

In [14]:
def process_group(df):
    dic = {}
    for name in ['time', 'creative_id', 'click_times', 'ad_id', 'product_id', 'product_category', 'advertiser_id', 'industry']:
        dic[name] = df[name].values
    return pd.Series(dic)

In [15]:
df_click_group = df_click.groupby('user_id').progress_apply(process_group)

100%|██████████| 4000000/4000000 [37:58<00:00, 1755.39it/s] 


## 将年龄和性别标签合并到df_train中

In [16]:
df_click_group = df_click_group.join(df_train_user.set_index('user_id'))

## 保存成pickle文件

In [24]:
df_click_group.dtypes

time                 object
creative_id          object
click_times          object
ad_id                object
product_id           object
product_category     object
advertiser_id        object
industry             object
age                 float64
gender              float64
dtype: object

In [28]:
df_click_group.to_pickle(os.path.join(save_path, 'processed_data_numerical.pkl'))